# Explore here

In [2]:
# Your code here
import pandas as pd

url = 'https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv'
total_data = pd.read_csv(url)

total_data.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [3]:
#Se transforma los datos categoricos a numericos
total_data["is_spam"] = total_data["is_spam"].apply(int)
total_data.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1


In [5]:
# Eliminar duplicados y reiniciar el índice
total_data = total_data.drop_duplicates().reset_index(drop=True)

# Obtener dimensiones del DataFrame
data_shape = total_data.shape
print(f"Dimensiones del DataFrame: {data_shape}")

# Calcular el número de elementos "spam" y "no spam"
spam_count = total_data[total_data['is_spam'] == 1].shape[0]
non_spam_count = total_data[total_data['is_spam'] == 0].shape[0]

# Imprimir los resultados
print(f"Spam: {spam_count}")
print(f"No spam: {non_spam_count}")

Dimensiones del DataFrame: (2369, 2)
Spam: 244
No spam: 2125


In [9]:
# Importar las bibliotecas necesarias
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk

# Descargar recursos de NLTK (si no están ya disponibles)
nltk.download('punkt')  # Tokenizador en inglés
nltk.download('stopwords')  # Lista de palabras vacías (stopwords)
nltk.download('wordnet')  # Base de datos para lematización

# Crear un conjunto de palabras vacías (stopwords) y un objeto para lematización
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Definir un conjunto con tokens irrelevantes que se encuentran comúnmente en URLs
# Estos incluyen elementos como "http", "https", "www" y extensiones de dominio
irrelevant_tokens = {'http', 'https', 'www', 'com', 'net', 'org'} 

# Función para procesar URLs
def preprocess_url(url):
    """
    Esta función toma una URL como entrada y la procesa para extraer tokens relevantes.
    1. Divide la URL utilizando delimitadores como '/', '.', '?' y '-'.
    2. Convierte los tokens en minúsculas y elimina aquellos que estén en la lista de tokens irrelevantes.
    3. Filtra palabras vacías (stopwords) de los tokens.
    4. Aplica lematización a los tokens para convertirlos a su forma base.
    """
    # Dividir la URL usando expresiones regulares
    tokens = re.split(r'[/.?-]+', url)  
    
    # Convertir tokens a minúsculas y eliminar tokens irrelevantes
    tokens = [token.lower() for token in tokens if token and token.lower() not in irrelevant_tokens]
    
    # Filtrar palabras vacías
    tokens = [token for token in tokens if token not in stop_words]
    
    # Aplicar lematización
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Retornar los tokens procesados
    return tokens

[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
# Aplicar la función de preprocesamiento a cada URL en la columna 'url'
# Esto transforma las URLs en conjuntos de tokens limpios y relevantes
total_data["url"] = total_data["url"].map(preprocess_url)

# Imprimir las primeras filas del DataFrame para inspeccionar los cambios
print("Primeras filas del DataFrame después del preprocesamiento de URLs:")
print(total_data.head())

Primeras filas del DataFrame después del preprocesamiento de URLs:
                                                 url  is_spam
0  [https:, briefingday, us8, list, manage, unsub...        1
1                                    [https:, hvper]        1
2                    [https:, briefingday, v4n3i4f3]        1
3  [https:, briefingday, n, 20200618, m#commentform]        0
4                         [https:, briefingday, fan]        1


In [11]:
# Importar la clase TfidfVectorizer desde la biblioteca scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

# Extraer la columna "url" del DataFrame y convertir los tokens en cadenas de texto unidas
# Cada lista de tokens se transforma en una única cadena con las palabras separadas por espacios
tokens_list = total_data["url"]
tokens_list = [" ".join(tokens) for tokens in tokens_list]  # Combinar tokens en texto para el análisis

# Configurar el vectorizador TF-IDF con parámetros específicos
# - max_features: máximo de características (palabras) a incluir
# - max_df: ignorar palabras que aparecen en más del 80% de los documentos
# - min_df: ignorar palabras que aparecen en menos de 5 documentos
vectorizer = TfidfVectorizer(max_features=5000, max_df=0.8, min_df=5)

# Transformar el texto en una matriz TF-IDF y convertirla en un arreglo numpy
X = vectorizer.fit_transform(tokens_list).toarray()

# Extraer la variable objetivo (etiquetas) del DataFrame
y = total_data["is_spam"]

# Imprimir información sobre las dimensiones de las matrices para ver el tamaño de los datos procesados
print(f"Matriz TF-IDF (X): {X.shape}")
print(f"Etiquetas (y): {y.shape}")

Matriz TF-IDF (X): (2369, 665)
Etiquetas (y): (2369,)


In [12]:
# Importar la función train_test_split para dividir los datos
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
# - test_size: Define el porcentaje de datos asignados al conjunto de prueba (20% en este caso)
# - random_state: Garantiza que la división sea reproducible si se ejecuta nuevamente
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir el tamaño de los conjuntos de entrenamiento y prueba para confirmar la división
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")


X_train shape: (1895, 665), X_test shape: (474, 665)
y_train shape: (1895,), y_test shape: (474,)


In [13]:
# Importar el modelo SVC (Support Vector Classifier) desde sklearn
from sklearn.svm import SVC

# Crear un modelo SVM con un núcleo RBF (Radial Basis Function)
# - kernel="rbf": Especifica que se usará la función base radial como kernel
# - C=1.0: Parámetro de regularización, un valor más alto penaliza más los errores
# - gamma=0.5: Define la influencia de cada punto de datos en la clasificación
model = SVC(kernel="rbf", C=1.0, gamma=0.5)

# Entrenar el modelo con los datos de entrenamiento
# X_train: Características (inputs) del entrenamiento
# y_train: Etiquetas (outputs) del entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones sobre los datos de prueba
# Esto genera un array con las clases predichas para cada ejemplo de prueba
y_pred = model.predict(X_test)


In [14]:
# Importar la función accuracy_score para calcular la precisión del modelo
from sklearn.metrics import accuracy_score

# Calcular la precisión comparando las etiquetas reales (y_test) con las predicciones (y_pred)
accuracy = accuracy_score(y_test, y_pred)

# Imprimir el resultado de la precisión con un formato claro
print(f"Precisión del modelo: {accuracy:.4f}")


Precisión del modelo: 0.9599


La precisión obtenida del 95% es un resultado bastante prometedor para este tipo de modelo. Sin embargo, existe la posibilidad de mejorar el rendimiento ajustando los hiperparámetros para buscar configuraciones más óptimas y perfeccionar el proceso de aprendizaje.

In [15]:
# Importar GridSearchCV para la búsqueda de hiperparámetros y SVC como modelo base
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'C': [0.1, 1, 10, 100],      # Valores del parámetro de regularización
    'gamma': [0.001, 0.01, 0.1, 1],  # Valores para el parámetro gamma (influencia de puntos)
    'kernel': ['rbf', 'linear']  # Tipos de kernel a probar
}

# Configurar el objeto GridSearchCV
# - refit=True: Reajusta el modelo utilizando los mejores parámetros encontrados
# - cv=5: Realiza validación cruzada con 5 particiones
# - verbose=2: Muestra información sobre el progreso del ajuste
grid_search = GridSearchCV(SVC(), param_grid, refit=True, cv=5, verbose=2)

# Entrenar GridSearchCV con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprimir los resultados del mejor conjunto de hiperparámetros
print("Mejores parámetros encontrados:", grid_search.best_params_)
print("Modelo óptimo:", grid_search.best_estimator_)

# Utilizar el mejor modelo encontrado para hacer predicciones en los datos de prueba
y_pred = grid_search.predict(X_test)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=   0.2s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=   0.2s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=   0.2s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=   0.2s
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time=   0.2s
[CV] END ..................C=0.1, gamma=0.001, kernel=linear; total time=   0.2s
[CV] END ..................C=0.1, gamma=0.001, kernel=linear; total time=   0.2s
[CV] END ..................C=0.1, gamma=0.001, kernel=linear; total time=   0.2s
[CV] END ..................C=0.1, gamma=0.001, kernel=linear; total time=   0.2s
[CV] END ..................C=0.1, gamma=0.001, kernel=linear; total time=   0.2s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.2s
[CV] END ......................C=0.1, gamma=0.0

In [16]:
accuracy_score(y_test, y_pred)

0.9641350210970464

Un aumento de precisión del 95.99% al 96.41% puede parecer pequeño en términos absolutos, pero en problemas complejos como la clasificación de spam, incluso una mejora del 0.42% es significativa. Este incremento podría indicar que la optimización de los hiperparámetros ha ayudado al modelo a capturar patrones más sutiles en los datos.